# IESB - CIA035 - Aula 15 - Malaria Transfer Learn

In [1]:
# Importando as bibliotecas
import os
import time

import numpy as np
import torch
import matplotlib.pyplot as plt
from torchvision import datasets, transforms, models
from torch.utils.data import random_split, DataLoader
from torch import nn, optim

In [2]:
# Verificando o diretório com as imagens
img_dir = "/kaggle/input/cell-images-for-detecting-malaria/cell_images/cell_images"
print(os.listdir(img_dir))

['Uninfected', 'Parasitized']


In [3]:
# Temos GPU?
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

cuda


## Data Augumentation

In [4]:
# Transformações que serão aplicadas às imagens
train_transform = transforms.Compose([transforms.Resize((224, 224)),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomVerticalFlip(),
                                      transforms.RandomRotation(35),
                                      transforms.RandomGrayscale(p=0.02),                                      
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])

In [5]:
# Criando nossos conjuntos de dados com base nas imagens
train = datasets.ImageFolder(img_dir, transform=train_transform)

In [6]:
# Dividindo os dados em treino e teste
train_data, test_data = random_split(train, (20000, 7558))

In [7]:
# Definindo os loaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True)

In [8]:
# Usando uma Resnet18 por meio de Transfer Learning
model = models.resnet18(pretrained=True)

model

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [9]:
# Vamos configurar a rede e redefinir a camada fully connected

# Congelando os parâmetros originais
for param in model.parameters():
    param.requires_grad = False

# Definindo o novo classificador
new_fc = nn.Linear(512, 2)

# Substituindo o classificador original
model.fc = new_fc

model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [10]:
# Enviando o modelo para GPU
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [11]:
# Temos que definir a função de erro e o otimizador (que vai alterar os pesos dos perceptrons)
error_function = nn.CrossEntropyLoss() # criterion
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

error_function.to(device)

CrossEntropyLoss()

In [12]:
# Treinamento do modelo

# Definindo o número de épocas
epochs = 10

# Colocando o modelo em modo de treinamento
model.train()


# For para rodar o número de épocas
for i in range(epochs):
    # Treinamento
    
    # Monitorando o training loss
    train_loss = 0.0
    
    # Obtendo dados e respostas
    for data, target in train_loader:
        
        # Enviando os dados para GPU, se existir
        data, target = data.to(device), target.to(device)
    
        # Foward Propagation (passando os dados de treino pela rede)
        outputs = model(data)
        # Calculando o erro
        loss = error_function(outputs, target)
       
        # Back Propagation
        # Limpar os parametros do otimizador (zerar o Gradiente Descendent)
        optimizer.zero_grad()
        # Calcular os novos pesos
        loss.backward()
        # Executar o optimizador (efetivamente fazer o back propagation mudando os pesos)
        optimizer.step()
        
        # Atualizando o training loss
        train_loss += loss.item() * data.size(0)
        
    # Calculando a média de erro por epoch
    train_loss = train_loss/len(train_loader.dataset)

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(i+1, train_loss))

Epoch: 1 	Training Loss: 0.331651
Epoch: 2 	Training Loss: 0.264127
Epoch: 3 	Training Loss: 0.247027
Epoch: 4 	Training Loss: 0.239607
Epoch: 5 	Training Loss: 0.237365
Epoch: 6 	Training Loss: 0.240054
Epoch: 7 	Training Loss: 0.236345
Epoch: 8 	Training Loss: 0.228911
Epoch: 9 	Training Loss: 0.234623
Epoch: 10 	Training Loss: 0.222412


In [13]:
# Variaveis para controlar os acertos das previsões da rede
# e  calcular a acurácia
correct = 0
total = 0

# Vamos colocar o modelo em modo de avaliação/teste
model.eval()

# Obtendo dados e respostas
for data, target in test_loader:
    
    # Enviando os dados para GPU, se existir
    data, target = data.to(device), target.to(device)    
    
    output = model(data)
    
    for index, i in enumerate(output):
        if torch.argmax(i) == target[index]:
            correct += 1
        total += 1

In [14]:
print('Accuracy: ', round(correct/total, 3))

Accuracy:  0.918
